In [19]:
"""
Summary:
Establishes an SSH connection to a remote computer and a connection to a MySQL database.
For each .MP4 file in a specified directory on the remote computer:
- Uses FFmpeg commands to obtain video properties.
- Uses file system commands to obtain the creation date and time of the video.
- Inserts these properties into the 'video' table of the MySQL database.
Finally, closes all open connections.

Dependencies:
- paramiko
- mysql.connector
"""

import paramiko
import mysql.connector
import os
import pandas as pd

# === Part 1: Connect to remote computer and database ===
# SSH connection parameters
hostname = 'gate.bss.phy.cam.ac.uk'
port = 22
username = 'yz655'
password = 'gDlzQXwR'

# Establish an SSH connection
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname, port, username, password)

# Establish a connection to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [20]:
# -----------------------------------------
# 2. Extract video data from the Excel file and populate the database
# -----------------------------------------
file_path = "C:\\Users\\46596\\Desktop\\Multi DDM\\manage_videos\\summary.xlsx"
df = pd.read_excel(file_path)

# Delete all existing entries from video table
cursor.execute("DELETE FROM video")
# cursor.execute("ALTER TABLE video AUTO_INCREMENT = 1;")

# Iterate through each row in the dataframe and populate the database
for index, row in df.iterrows():
    
    create_time = row['Created Time'].strftime('%H:%M:%S')
    
    duration_parts = row['Duration'].split(':')
    duration_seconds = float(duration_parts[0]) * 3600 + float(duration_parts[1]) * 60 + float(duration_parts[2])

    video_properties = {
        "filename": row["FileName"],
        "location": "/cicutagroup/crop_data/wheat_videos/drone_videos/original/" + row["FileName"],
        "create_date": row["Created Date"],
        "create_time": row['Created Time'],
        "duration": row['Duration'],
        "format": "MP4",
        "frame_width": row["Width"],
        "frame_height": row["Height"],
        "frame_rate": row["Framerate"],
        "frame_number": row["NumFrames"],
        "color_space": row["Color MD"]
    }

    add_video = ("INSERT INTO video "
                 "(filename, location, create_date, create_time, duration, format, frame_width, frame_height, frame_rate, frame_number, color_space) "
                 "VALUES (%(filename)s, %(location)s, %(create_date)s, %(create_time)s, %(duration)s, %(format)s, %(frame_width)s, %(frame_height)s, %(frame_rate)s, %(frame_number)s, %(color_space)s)")

    cursor.execute(add_video, video_properties)
    cnx.commit()
    print(f"Entry added for video: {row['FileName']}")

Entry added for video: DJI_0004.MP4
Entry added for video: DJI_0012.MP4
Entry added for video: DJI_0013.MP4
Entry added for video: DJI_0014.MP4
Entry added for video: DJI_0015.MP4
Entry added for video: DJI_0016.MP4
Entry added for video: DJI_0017.MP4
Entry added for video: DJI_0018.MP4
Entry added for video: DJI_0019.MP4
Entry added for video: DJI_0024.MP4
Entry added for video: DJI_0025.MP4
Entry added for video: DJI_0026.MP4
Entry added for video: DJI_0028.MP4
Entry added for video: DJI_0029.MP4
Entry added for video: DJI_0030.MP4
Entry added for video: DJI_0031.MP4
Entry added for video: DJI_0032.MP4
Entry added for video: DJI_0033.MP4
Entry added for video: DJI_0034.MP4
Entry added for video: DJI_0035.MP4
Entry added for video: DJI_0036.MP4
Entry added for video: DJI_0037.MP4
Entry added for video: DJI_0038.MP4
Entry added for video: DJI_0039.MP4
Entry added for video: DJI_0040.MP4
Entry added for video: DJI_0041.MP4
Entry added for video: DJI_0042.MP4
Entry added for video: DJI_0

In [ ]:
import os
import mysql.connector
import paramiko
import re

# # 1. Connect to remote computer and database
# hostname = 'gate.bss.phy.cam.ac.uk'
# port = 22
# username = 'yz655'
# password = 'gDlzQXwR'

# ssh_client = paramiko.SSHClient()
# ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# ssh_client.connect(hostname=hostname, port=port, username=username, password=password)

base_dir = "../../../cicutagroup/crop_data/wheat_videos/drone_videos/"
folders_to_check = ["rescale_crop_fps10", "crop_fps10", "fps10", "stabilized_rescale_crop_fps10"]

# Manual mapping of video properties for each folder
folder_video_properties = {
    "fps10": {"frame_width": 3840, "frame_height": 2160, "frame_rate": 10, "color_space": "YUV"},
    "crop_fps10": {"frame_width": 2160, "frame_height": 2160, "frame_rate": 10, "color_space": "YUV"},
    "rescale_crop_fps10": {"frame_width": 1024, "frame_height": 1024, "frame_rate": 10, "color_space": "YUV"},
    "stabilized_rescale_crop_fps10": {"frame_width": 1024, "frame_height": 1024, "frame_rate": 10, "color_space": "YUV"}
}

# cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
# cursor = cnx.cursor()

for folder in folders_to_check:
    directory = os.path.join(base_dir, folder)
    stdin, stdout, stderr = ssh_client.exec_command(f"ls {directory}")
    files = stdout.read().decode().splitlines()

    for file in files:
        
        # Check if video already exists in the processed_videos table
        cursor.execute(f"SELECT 1 FROM processed_videos WHERE filename='{file}'")
        if cursor.fetchone():
            continue  # skip this video if it's already in the database

        # Extract the original video name using regex
        original_file_name_match = re.search(r'DJI_\d{4}\.MP4', file)
        if not original_file_name_match:
            continue  # skip this file if it doesn't match the pattern
        original_file_name = original_file_name_match.group()
        
        # Fetch details from the original video entry
        cursor.execute(f"SELECT video_id, create_date, create_time, duration FROM video WHERE filename='{original_file_name}'")
        video_id, create_date, create_time, duration = cursor.fetchone()

        # Convert create_time to "hh:mm:ss" format
        total_seconds = create_time.total_seconds()
        hours = int(total_seconds // 3600)
        minutes = int((total_seconds % 3600) // 60)
        seconds = int(total_seconds % 60)
        formatted_time = "{:02}:{:02}:{:02}".format(hours, minutes, seconds)
        
        video_properties = {
            "video_id": video_id,
            "filename": file,
            "location": os.path.join(directory, file),
            "create_date": create_date,
            "create_time": formatted_time, # use the formatted time
            "duration": duration.total_seconds(), # convert timedelta to total seconds
            "format": "MP4",
            "frame_width": folder_video_properties[folder]["frame_width"],
            "frame_height": folder_video_properties[folder]["frame_height"],
            "frame_rate": folder_video_properties[folder]["frame_rate"],
            "color_space": folder_video_properties[folder]["color_space"],
            "processing_type": folder
        }
        
        # Insert into the 'processed_videos' table
        add_video = ("INSERT INTO processed_videos "
                    "(video_id, filename, location, create_date, create_time, duration, format, frame_width, frame_height, frame_rate, color_space, processing_type) "
                    "VALUES (%(video_id)s, %(filename)s, %(location)s, %(create_date)s, %(create_time)s, %(duration)s, %(format)s, %(frame_width)s, %(frame_height)s, %(frame_rate)s, %(color_space)s, %(processing_type)s)")
        cursor.execute(add_video, video_properties)
        cnx.commit()


# # 3. Close connection
# cursor.close()
# cnx.close()
# ssh_client.close()


In [11]:
import os
import mysql.connector
import pandas as pd
import re
import numpy as np

# Step 2: Read the summary Excel file
summary_df = pd.read_excel("C:\\Users\\46596\\Desktop\\Multi DDM\\manage_videos\\summary.xlsx")
summary_excel_path = "C:\\Users\\46596\\Desktop\\Multi DDM\\manage_videos\\summary.xlsx"

# Define function to get video_id
def get_video_id(filename):
    cursor.execute(f"SELECT video_id FROM video WHERE filename='{filename}'")
    return cursor.fetchone()[0]

# Define function to get processed_video_id
def get_processed_video_id(filename):
    cursor.execute(f"SELECT processed_video_id FROM processed_videos WHERE filename='{filename}'")
    result = cursor.fetchone()
    return result[0] if result else None

def get_video_summary_from_excel(filename, video_name):
    df = pd.read_excel(filename)
    video_summary = df[df["FileName"] == video_name].iloc[0].to_dict()

    # Convert numpy types to native Python types
    for key, value in video_summary.items():
        if isinstance(value, (np.int64, np.int32)):
            video_summary[key] = int(value)
        elif isinstance(value, np.float64):
            video_summary[key] = float(value)
        else:
            continue

    return video_summary


# Step 3: Iterate through the files
analysis_directory = "../../../cicutagroup/crop_data/wheat_videos/drone_videos/stabilized_drone_videos_to_run_Analysis"
# directory = os.path.join(base_dir, folder)
stdin, stdout, stderr = ssh_client.exec_command(f"ls {analysis_directory}")
files = stdout.read().decode().splitlines()
    
for file in files:
    if file.endswith(".mat"):
        # Extract the original video name using regex
        original_file_name_match = re.search(r'DJI_\d{4}.mat', file)
        if original_file_name_match:
            original_file_name = original_file_name_match.group().replace('.mat', '.MP4')
            # Extract additional information
            video_id = get_video_id(original_file_name)
            processed_video_id = get_processed_video_id(file)
            
            # Step 4: Extract details from the Excel summary
            # video_summary = summary_df[summary_df["FileName"] == original_file_name].iloc[0]
            # Get video summary from Excel
            video_summary = get_video_summary_from_excel(summary_excel_path, original_file_name)


            # Step 5: Insert into the 'ddm_data' table
            ddm_data_entry = {
                "video_id": video_id,
                "processed_video_id": processed_video_id,
                "DDM_data_file_location": os.path.join(analysis_directory, file),
                "DDM_data_nonAvgIqtau_file_location": None,
                "DDM_data_nonAvgIqtau_angle_Iqtau_file_location": None,
                "MedianFrequencyVec": video_summary["MedianFrequencyVec"],
                "Major_q": video_summary["Major_q"],
                "Major_q_amplitude": video_summary["Major_q_amplitude"],
                "Major_q_frequency": video_summary["Major_q_frequency"],
                "Major_q_damping": video_summary["Major_q_damping"],
                "Major_q_offset": video_summary["Major_q_offset"],
                "Major_q_gof": video_summary["Major_q_gof"]
            }

            add_ddm_data = ("INSERT INTO ddm_data "
                            "(video_id, processed_video_id, DDM_data_file_location, DDM_data_nonAvgIqtau_file_location, "
                            "DDM_data_nonAvgIqtau_angle_Iqtau_file_location, MedianFrequencyVec, Major_q, Major_q_amplitude, "
                            "Major_q_frequency, Major_q_damping, Major_q_offset, Major_q_gof) "
                            "VALUES (%(video_id)s, %(processed_video_id)s, %(DDM_data_file_location)s, "
                            "%(DDM_data_nonAvgIqtau_file_location)s, %(DDM_data_nonAvgIqtau_angle_Iqtau_file_location)s, "
                            "%(MedianFrequencyVec)s, %(Major_q)s, %(Major_q_amplitude)s, %(Major_q_frequency)s, "
                            "%(Major_q_damping)s, %(Major_q_offset)s, %(Major_q_gof)s)")

            cursor.execute(add_ddm_data, ddm_data_entry)
            cnx.commit()

# cursor.close()
# cnx.close()


Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [15]:
# 2. Use the ALTER TABLE command to add the filename column
try:
    cursor.execute("ALTER TABLE ddm_data ADD filename VARCHAR(255)")
    print("Added filename column.")
except mysql.connector.Error as err:
    if "duplicate column name" in str(err).lower():
        print("Column 'filename' already exists.")
    else:
        print(f"Error: {err}")
        cursor.close()
        cnx.close()
        exit()

# 3. Update the filename values
cursor.execute("SELECT DDM_data_id, DDM_data_file_location FROM ddm_data")
for ddm_data_id, file_location in cursor.fetchall():
    filename = os.path.basename(file_location)
    cursor.execute("UPDATE ddm_data SET filename = %s WHERE DDM_data_id = %s", (filename, ddm_data_id))

# Commit the changes
cnx.commit()

Added filename column.


In [18]:

# Fetch all entries from the ddm_data table
cursor.execute("SELECT DDM_data_id, filename FROM ddm_data WHERE processed_video_id IS NULL")
entries = cursor.fetchall()

for entry_id, filename in entries:
    # Extract the original filename from the DDM data file name
    original_file_name = filename.replace('.mat', '.MP4')

    # Fetch processed_video_id from the processed_videos table for the corresponding video
    cursor.execute(f"SELECT processed_video_id FROM processed_videos WHERE filename='{original_file_name}'")
    processed_video_id_row = cursor.fetchone()

    # If a match is found, update the ddm_data entry with the processed_video_id
    if processed_video_id_row:
        processed_video_id = processed_video_id_row[0]
        cursor.execute(f"UPDATE ddm_data SET processed_video_id={processed_video_id} WHERE DDM_data_id={entry_id}")
        cnx.commit()


Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [ ]:
# # === Part 2: Enter the video entry into the database ===

# # List all .MP4 files in the directory
# command = "ls ../../../cicutagroup/crop_data/wheat_videos/drone_videos/original/*.MP4"
# stdin, stdout, stderr = ssh_client.exec_command(command)
# files = stdout.readlines()

# for file in files:
#     file = file.strip()
    
#     # Using FFprobe to get video properties
#     ffprobe_cmd = f"ffprobe -v error -select_streams v:0 -show_entries stream=width,height,r_frame_rate,duration,nb_frames,pix_fmt -of csv=p=0 {file}"
#     stdin, stdout, stderr = ssh_client.exec_command(ffprobe_cmd)
#     video_info = stdout.read().decode().split(',')
    
#     # Use 'stat' to get the file's creation date and time
#     date_command = f"stat -c %y {file}"
#     stdin, stdout, stderr = ssh_client.exec_command(date_command)
#     date_output = stdout.read().decode().strip()
#     create_date, create_time = date_output.split()[0], date_output.split()[1].split('.')[0]

#     video_properties = {
#         "filename": os.path.basename(file),
#         "location": file,
#         "create_date": create_date,
#         "create_time": create_time,
#         "duration": video_info[3],
#         "format": "MP4",
#         "frame_width": video_info[0],
#         "frame_height": video_info[1],
#         "frame_rate": float(video_info[2].split('/')[0]) / float(video_info[2].split('/')[1]),
#         "frame_number": video_info[4],
#         "color_space": video_info[5]
#     }
    
#     # SQL command to insert the data
#     add_video = ("INSERT INTO video "
#                  "(filename, location, create_date, create_time, duration, format, frame_width, frame_height, frame_rate, frame_number, color_space) "
#                  "VALUES (%(filename)s, %(location)s, %(create_date)s, %(create_time)s, %(duration)s, %(format)s, %(frame_width)s, %(frame_height)s, %(frame_rate)s, %(frame_number)s, %(color_space)s)")
    
#     # Execute the command and commit to save changes
#     cursor.execute(add_video, video_properties)
#     cnx.commit()
    
#     # Print a message indicating an entry has been added
#     print(f"Entry added for video: {video_properties['filename']}")


In [3]:
# Retrieve all video entries
query = "SELECT * FROM video;"
cursor.execute(query)

# Print the entries
print("Video Entries:")
print("---------------------------------------------------------------------------------------------")
for entry in cursor:
    print(entry)

Video Entries:
---------------------------------------------------------------------------------------------
(1, 'DJI_0004.MP4', '/cicutagroup/crop_data/wheat_videos/drone_videos/original/DJI_0004.MP4', datetime.date(2022, 7, 9), datetime.timedelta(seconds=44650), datetime.timedelta(seconds=54), 'MP4', 1024, 1024, 10.0, 546, 'default', None, None, None, None, None, None, None)
(2, 'DJI_0012.MP4', '/cicutagroup/crop_data/wheat_videos/drone_videos/original/DJI_0012.MP4', datetime.date(2022, 7, 9), datetime.timedelta(seconds=49874), datetime.timedelta(seconds=11), 'MP4', 1024, 1024, 10.0, 111, 'default', None, None, None, None, None, None, None)
(3, 'DJI_0013.MP4', '/cicutagroup/crop_data/wheat_videos/drone_videos/original/DJI_0013.MP4', datetime.date(2022, 7, 9), datetime.timedelta(seconds=49898), datetime.timedelta(seconds=11), 'MP4', 1024, 1024, 10.0, 113, 'default', None, None, None, None, None, None, None)
(4, 'DJI_0014.MP4', '/cicutagroup/crop_data/wheat_videos/drone_videos/original

In [3]:
# Retrieve column names for the 'video' table
query = """
SELECT COLUMN_NAME
FROM information_schema.COLUMNS
WHERE TABLE_SCHEMA = 'crop_videos' AND TABLE_NAME = 'ddm_data';
"""
cursor.execute(query)

# Print the column names
print("Columns in 'video' table:")
for (column_name,) in cursor:
    print(column_name)

Columns in 'video' table:
DDM_data_id
video_id
processed_video_id
DDM_data_file_location
DDM_data_nonAvgIqtau_file_location
DDM_data_nonAvgIqtau_angle_Iqtau_file_location
MedianFrequencyVec
Major_q
Major_q_amplitude
Major_q_frequency
Major_q_damping
Major_q_offset
Major_q_gof


In [16]:
# Insert entries into video_group table
video_group_entries = [
    ("DJI_videos_1", "Pietro Cicuta"),
    ("DJI_videos_2", "Yuyi Zhang")
]

insert_video_group = "INSERT INTO video_group (group_name, creator) VALUES (%s, %s)"
cursor.executemany(insert_video_group, video_group_entries)

# Insert entries into operator table
operator_entries = [
    ("Pietro Cicuta",),
    ("Yuyi Zhang",)
]

insert_operator = "INSERT INTO operator (operator_name) VALUES (%s)"
cursor.executemany(insert_operator, operator_entries)

# Insert entry into device table
insert_device = "INSERT INTO device (device_type) VALUES (%s)"
cursor.execute(insert_device, ("DJI Mavic 3",))

# Commit the changes
cnx.commit()

Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [22]:
# Fetch current frame_number for all entries
cursor.execute("SELECT video_id, frame_number FROM video")
results = cursor.fetchall()

# Update the entries as required
for row in results:
    video_id = row[0]
    frame_number = row[1]
    new_frame_number = frame_number / 10 * 59.94

    update_query = """
        UPDATE video 
        SET frame_width = 3840, 
            frame_height = 2160, 
            frame_rate = 59.94, 
            frame_number = %s
        WHERE video_id = %s
    """
    cursor.execute(update_query, (new_frame_number, video_id))


In [ ]:
# === Part 3: Close connection ===
cursor.close()
cnx.close()
ssh_client.close()